# Исследование надежности заемщиков


Изучим общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Обработайте значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Этот столбец не понадобится вам для исследования.

Выведем перечень уникальных значений столбца `children`.**

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [15]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

 Обработаем неявные дубликаты в столбце `education`. 

In [16]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()

### Категоризация данных

 На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`

In [19]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)


Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.



In [74]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [76]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)


Исследуем данные 

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [48]:
df = pd.DataFrame()
df['ch_count'] = data.groupby('children')['debt'].agg('count') #количество оформленных кредитов, сгруппированных по колич детей
df['ch_sum'] = data.groupby('children')['debt'].agg('sum') #количество непогашенных кредитов, сгруппированных по колич детей 
df['ch_result%'] = (1 - df['ch_sum'] / df['ch_count'])*100 #доля  возвращенных кредитов к общему числу заемщиков, сгруппированных по колич детей 
df.sort_values('ch_result%', ascending=False) 

,ch_count,ch_sum,ch_result%
children,,,
5,9,0,100.000000
0,14091,1063,92.456178
3,330,27,91.818182
1,4808,444,90.765391
2,2052,194,90.545809
4,41,4,90.243902


**Вывод:                                                                                                                        
    По полученным данным видна зависимость возвратности кредитов от количества детей у заемщика. Так, больше всего кредитов берут бездетные, а меньше всего многодетные(3-5 детей) из-за количества расходов. "Лучшей" по возвратности также остается категория бездетных, однако, на втором месте находятся заемщики с 3-мя детьми (предположительно потому, что эта категория заемщиков стремиться иметь хорошую кредитную историю для  получения лучших процентный ставкок по ипотеке и другим крупный займам
   ).
    Также можно заметить, что заемщики с 3 детьми составляют маленькую долю из числа всей выборки, это может быть из-за отсутсвия денег на обслуживание кредитов, а в банк за кредитом из этой категории идут люди более зрелые и ответсвенные, имеющие хорошую должность и стабильный доход. Про заемщиков с 5-ю детьми действует та же логика, но данных в этой категории мало, чтобы сделать более полный вывод.    
    В категориях с 1 и 2 детьми возвратность уменьшается и является примерно равной, т.к. чаще люди с 1-2  детьми молодые,  и  не успели достигнуть карьерных высот, менее зрелые и не всегда умеют управлять бюджетом.
** 

 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
family = pd.DataFrame()
family['status_count'] = data.groupby('family_status')['debt'].agg('count') #количество оформленных кредитов, сгруппированных по семейному положению
family['status_sum'] = data.groupby('family_status')['debt'].agg('sum') #количество непогашенных кредитов, сгруппированных по семейному положению
family['fam_result%'] = (1 - family['status_sum'] / family['status_count'])*100 #доля  возвращенных кредитов к общему числу заемщиков, сгруппированных по семейному положению 
family.sort_values('fam_result%', ascending=False) 


,status_count,status_sum,fam_result%
family_status,,,
вдовец / вдова,951,63,93.375394
в разводе,1189,84,92.935240
женат / замужем,12261,927,92.439442
гражданский брак,4134,385,90.686986
Не женат / не замужем,2796,273,90.236052


**Вывод:                                                                                                                          Есть связь. Лучше всего возвращают займы вдовы, второе место занимают разведенные. Одинокие, бывшие в браке люди рассчитывают только на себя и лучше управляют личными финансами,возможно. На третьем месте заемщики, находящиеся в зарегистрированном браке.На эту категори приходится наибольшее число займов, люди создают условия для семьи. ** 

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [53]:
income = pd.DataFrame()
income['inc_count'] = data.groupby('total_income_category')['debt'].agg('count') #количество оформленных кредитов, сгруппированных по  фин положению
income['inc_sum'] = data.groupby('total_income_category')['debt'].agg('sum') #количество непогашенных кредитов, сгруппированных по фин положению
income['inc_result%'] = (1 - income['inc_sum'] / income['inc_count'])*100 #доля  возвращенных кредитов к общему числу заемщиков, сгруппированных по фин положению 
income.sort_values('inc_result%', ascending=False) 


,inc_count,inc_sum,inc_result%
total_income_category,,,
D,349,21,93.982808
B,5014,354,92.939769
A,25,2,92.000000
C,15921,1353,91.501790
E,22,2,90.909091


**Вывод:                                                                                                                       
                                                                                                                                   Зависимость есть. Заемщики из группы "В" (200001-1000000) возвращают кредиты лучше остальных, на втором месте наиболее многочисоленная группа с доходом "С" (50001-200000), в остальных группах недостаточно данных для сравнения. Из таблицы видим, что величина дохода влияет на возвращаемость и на количество займов
                                                                                                                                   ** 

 Как разные цели кредита влияют на его возврат в срок?

In [89]:
purpose = pd.DataFrame()
purpose['pur_count'] = data.groupby('purpose_category')['debt'].agg('count') #количество оформленных кредитов, сгруппированных по цели кредита
purpose['pur_sum'] = data.groupby('purpose_category')['debt'].agg('sum') #количество непогашенных кредитов, сгруппированных по цели кредита
purpose['pur_result%'] = (1 - purpose['pur_sum'] / purpose['pur_count'])*100 #доля  возвращенных кредитов к общему числу заемщиков, сгруппированных по цели кредита
purpose.sort_values('pur_result%', ascending=False)

,pur_count,pur_sum,pur_result%
purpose_category,,,
операции с недвижимостью,10751,780,92.744861
проведение свадьбы,2313,183,92.088197
получение образования,3988,369,90.747242
операции с автомобилем,4279,400,90.652022


**Вывод:                                                                                                                                     
           Цель кредита также влияет на возвращаемость займа. Так, по займам связаным с недвижимостью возвратность самая лучшая, возможно, из-за обязательных страховок по случаю потери заработка и др. Самый низкий процент возвращаемости в категории кредитов на автомобили, вероятно это связано с появлением неожиданных доп расходов на обслуживане автомобиля. ** 

Общий вывод.

*В итоге каждый фактор в будущей модели скоринга имеет свое влияние на способность потенциального заёмщика вернуть кредит банку. По результатам исследования можно увидеть, что самыми надежными заемщиками, с наиболее высокой долей возвращаемости кредитов, будут люди, подходящие под категории "Бездетные", "Вдовы/вдовцы", "с доходом В" (200001-1000000),"С" (50001-200000), "Операции с недвижимостью". И напротив, самыми неблагонадежными будут заемщики из категорий "с детьми", "не состоявшие в браке", "операции с автомобилем". Однако, невозможно достоверно сказать есть ли зависимосоть между уровнем дохода и возвратом кредита в срок из-за недостаточного количества данных по разным категорям, а также из-за слишком отличающихся интервалов в категориях дохода.*